In [1]:
import os
from IPython.display import clear_output
import requests
import re
import json
from bs4 import BeautifulSoup
caracSpe = ''
aRemplacer = r'[<>:"/\\|?*#\[\]]'

## récupération des données

In [2]:
pays = {}
rep = requests.get('https://emojiterra.com/fr/drapeaux-nationaux/')
if rep.ok:
  soup = BeautifulSoup(rep.text, 'lxml')
  listePays = soup.find('div', class_='archive-child')
  for a in listePays.findAll('a'):
    pays[a.find('span').text.strip()] = a['title'][re.search(':', a['title']).end():].strip()
else:
  print("problème à la page " + link)
  print(rep)
print('liste pays ok')

liste pays ok


### Liste des Episodes

In [3]:
linksEp = []
for i in range(1,4):
  url = "https://lddm.psln.nl/podcast/show/03495e3e-7b43-4bd6-9c2b-92d3b0532132?page=" + str(i)
  rep = requests.get(url)
  if rep.ok:
    soup = BeautifulSoup(rep.text, 'lxml')
    aList = soup.findAll('a', class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
    for a in aList:
      link = a['href']
      linksEp.append(link)
  else:
    print("problème à la page " + i)
    print(rep)
print("liste lien episodes (linksEp) : Ok")

liste lien episodes (linksEp) : Ok


In [22]:
episodes = {}
boucle = 0
ouManque = []
manqueJeu = 0
manqueDate = 0
manqueStudio = 0
for ep in linksEp:
#for i in range(1):
  #ep = 'https://lddm.psln.nl/episode/show/9e761534-1332-4b32-84a9-2d43d74fe38c'
  rep = requests.get(ep)
  if rep.ok:
    soup = BeautifulSoup(rep.text, 'lxml')
    titre = re.sub(aRemplacer, caracSpe, soup.find('h1', class_="card-title text-center").text)
    metadata = soup.findAll('li', class_="list-inline-item")
    nSaison = re.search("\d+",metadata[0].text).group()
    nEpisode = re.search("\d+",metadata[1].text).group()
    datePubli = re.search("\S+",metadata[2].text).group()
    duree = re.search("\S+",metadata[3].text).group()
    text = re.sub(r"\s+", " ", soup.find("p", class_="p-4 text-justify").text.strip())
    trackList = soup.findAll("div", class_="list-group list-group-flush")
    tracks = trackList[1].findAll("a", class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
    trackListe = []
    for a in tracks:
      track = {'link' : a['href'],
              'name' : re.sub(aRemplacer, caracSpe, a.find('strong').text.strip()),
              'index' : re.search("\d+",a.text).group(),
               'reprise' : a.find('span', class_='badge badge-secondary badge-pill').text.strip() == 'Reprise'
              }
      repArt = requests.get(track['link'])
      if repArt.ok:
        track['artist'] = []
        for art in BeautifulSoup(repArt.text, 'lxml').find('ol', class_='list-inline text-center').findAll('li', class_='list-inline-item'):
          track['artist'].append(re.sub(aRemplacer, caracSpe, art.find('a').text.strip()))
      else:
        print("problème à la page " + track['link'])
        print(repArt)
      if re.search(r"\,\s+\d{4}\.",a.text):
        track['dateJeu'] = re.search("\d{4}",re.search(r"\,\s+\d{4}\.",a.text).group()).group()
      else:
        track['dateJeu'] = ''
        manqueDate += 1
        ouManque.append([track['name'], titre, ep])
      if re.search('\s+.+\,\s+<em>',str(a)):
        track['studio'] = re.sub(aRemplacer, caracSpe, re.search('\s.+', re.sub('\s+', ' ', re.search('\s+.+\,\s+<em>',str(a)).group()).strip()).group()[1:-6])
      else:
        track['studio'] = ''
        manqueStudio += 1
      if a.find('em'):
        track['game'] = re.sub(aRemplacer, caracSpe, a.find('em').text.strip())
      else:
        track['game'] = ''
        manqueJeu += 1
      trackListe.append(track)
    clear_output(wait=True)
    boucle += 1
    print(str(boucle) + '/87')
    episode = {'titre' : titre,
              'link' : ep,
              'nSaison' : nSaison,
              'nEpisode' : nEpisode,
              'datePubli' : datePubli,
              'duree' : duree,
              'text' : text,
              'trackListe' : trackListe
              }
    episodes[str(nEpisode)] = episode
  else:
    print("problème à la page " + ep)
    print(rep)
with open('C:/Users/Jayma/Documents/Jupyter notebook/Json LDDM to Obsidian/Episodes.json', 'w') as outfile:
    json.dump(episodes, outfile)
print('il manque ' + str(manqueJeu) + ' nom de jeu, ' + str(manqueDate) + 'date de jeu, et ' + str(manqueStudio) + 'nom du studio.')
print('données des épisodes sauvegardé (Episodes.json)')

87/87
il manque 27 nom de jeu, 27date de jeu, et 27nom du studio.
données des épisodes sauvegardé (Episodes.json)


### Liste des Artistes

In [23]:
linksArt = []
for i in range(1,55):
  url = "https://lddm.psln.nl/contact?page=" + str(i)
  rep = requests.get(url)
  if rep.ok:
    soup = BeautifulSoup(rep.text, 'lxml')
    aList = soup.findAll('a', class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
    aList = aList[2:]
    for a in aList:
      link = a['href']
      linksArt.append(link)
  else:
    print("problème à la page " + i)
    print(rep)
print("liste lien artistes (linksArt) : Ok")

liste lien artistes (linksArt) : Ok


In [24]:
artistes = {}
boucle = 0
for link in linksArt:
  rep = requests.get(link)
  if rep.ok:
    soup = BeautifulSoup(rep.text, 'lxml')
    nomArt = re.sub(aRemplacer, caracSpe, soup.find('h1', class_="card-title text-center").text).strip()
    metadata = soup.findAll('li', class_="list-inline-item")
    nomNom = re.sub(r"\s+", " ", metadata[0].text.strip())
    if not metadata[2].text.strip() == 'Lieu de travail inconnu':
      nationalite = pays[metadata[2].text.strip()[:re.search('\s', metadata[2].text.strip()).start()]]
    else:
      nationalite = ''
    if re.search("\d+\/\d+\/\d+",metadata[3].text):
      dateNaisance = re.search("\d+\/\d+\/\d+",metadata[3].text).group()
    else:
      dateNaisance = ""
    if soup.find('i', class_="fas fa-mars"):
      sexe = soup.find('i', class_="fas fa-mars")['title']
    else:
      sexe = ""
    listes = soup.findAll("div", class_="list-group list-group-flush")
    trackList = []
    listNbr = 0
    if soup.find('h2', class_='h5 card-header', text= 'Liste des pistes'):
      listNbr = 1
      tracks = listes[0].findAll("a", class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
      for a in tracks:
        track = {'link' : re.sub(r"\/episode\/", "\/track\/", a['href']),
                'name' : a.find('em').text.strip(),
                'nameEp' : re.sub(aRemplacer, caracSpe, a.find('strong').text.strip()),
                'indexEp' : re.search("\d+",a.text).group()
                }
        trackList.append(track)
    sourceListe = []
    if soup.find('h2', class_='h5 card-header', text='Liste des sources'):
      sources = listes[listNbr].findAll("a", class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
      for a in sources:
        source = {
            'nom' : a.find('span').text.strip(),
            'link' : a['href']
        }
        sourceListe.append(source)
    artiste = {
        'nomArt' : nomArt,
        'link' : link,
        'nomNom' : nomNom,
        'nationalite' : nationalite,
        'dateNaisance' : dateNaisance,
        'sexe' : sexe,
        'trackList' : trackList,
        'sourceListe' : sourceListe
        }
    clear_output(wait=True)
    boucle += 1
    print(boucle, '/1601')
    artistes[nomArt] = artiste
  else:
    print("problème à la page " + link)
    print(rep)


with open('C:/Users/Jayma/Documents/Jupyter notebook/Json LDDM to Obsidian/Artistes.json', 'w') as outfile:
    json.dump(artistes, outfile)
print('données des artistes sauvegardé (Artistes.json)')

1601 /1601
données des artistes sauvegardé (Artistes.json)


### Liste des Jeux

In [25]:
linksGame = []
for i in range(1,51):
  url = "https://lddm.psln.nl/game?page=" + str(i)
  rep = requests.get(url)
  if rep.ok:
    soup = BeautifulSoup(rep.text, 'lxml')
    aList = soup.findAll('a', class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
    aList = aList[2:]
    for a in aList:
      link = a['href']
      linksGame.append(link)
  else:
    print("problème à la page " + i)
    print(rep)
print("liste lien jeux (linksGame) : Ok")

liste lien jeux (linksGame) : Ok


In [26]:
jeux = {}
jeuDoublon = []
lienErreur = []
paysManque = {}
nErreur = 0
boucle = 0
for link in linksGame:
#for i in range(2):
  #link = linksGame[i]
  rep = requests.get(link)
  if rep.ok:
    soup = BeautifulSoup(rep.text, 'lxml')
    nomJeu = re.sub(aRemplacer, caracSpe, soup.find('h1', class_="card-title text-center").text)
    metadata = soup.findAll('ol', class_="list-inline text-center")
    if metadata[0].find('li').text.strip():
      datePubli = metadata[0].find('li').text.strip()
    else:
      datePubli = ""
    studio = re.sub(aRemplacer, caracSpe, metadata[1].text.strip())[re.search('\s', re.sub(aRemplacer, caracSpe, metadata[1].text.strip())).end():].strip()
    if re.sub(aRemplacer, caracSpe, metadata[1].text.strip())[:re.search('\s', re.sub(aRemplacer, caracSpe, metadata[1].text.strip())).start()] in pays:
      nat = pays[re.sub(aRemplacer, caracSpe, metadata[1].text.strip())[:re.search('\s', re.sub(aRemplacer, caracSpe, metadata[1].text.strip())).start()]]
    else:
      paysManque[nomJeu] = re.sub(aRemplacer, caracSpe, metadata[1].text.strip())[:re.search('\s', re.sub(aRemplacer, caracSpe, metadata[1].text.strip())).start()]
    studioLink = metadata[1].find('a')['href']
    modeJeu = []
    for mode in metadata[2].findAll('li'):
      modeJeu.append(mode.text.strip())
    genres = []
    for genre in metadata[3].findAll('li'):
      genres.append(genre.text.strip())
    consoles = []
    for console in metadata[4].findAll('li'):
      consoles.append(console.text.strip())
    listes = soup.findAll("ul", class_="list-group list-group-flush")
    trackList = []
    listNbr = 0
    if soup.find('h2', class_='h5 card-header', text= 'Liste des pistes'):
      listNbr = 1
      tracks = listes[0].findAll("a", class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
      for a in tracks:
        track = {'link' : a['href'],
                'name' : a.find('strong').text.strip(),
                'indexEp' : re.search('\d+', re.search("\s{10}\d{1,2}\.\s",a.text).group()).group()
                }
        trackList.append(track)
    sourceListe = []
    if soup.find('h2', class_='h5 card-header', text='Liste des sources'):
      sources = listes[listNbr].findAll("a", class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
      for a in sources:
        source = {
            'nom' : a.find('span').text.strip(),
            'link' : a['href']
        }
        sourceListe.append(source)
    jeu = {
        'nomJeu' : nomJeu,
        'link' : link,
        'datePubli' : datePubli,
        'studio' : studio,
        'studioLink' : studioLink,
        'nat' : nat,
        'modeJeu' : modeJeu,
        'genres' : genres,
        'consoles' : consoles,
        'trackList' : trackList,
        'sourceListe' : sourceListe
        }
    clear_output(wait=True)
    boucle += 1
    print(boucle, '/1485')
    if nomJeu in jeux.keys():
      jeuDoublon.append(nomJeu)
      jeu['nomJeu'] = nomJeu+ ' (2)'
      jeux[nomJeu + '(2)'] = jeu
    else:
      jeux[nomJeu] = jeu
  else:
    nErreur += 1
    lienErreur.append(link)
    print("problème à la page " + link)
    print(rep)


with open('C:/Users/Jayma/Documents/Jupyter notebook/Json LDDM to Obsidian/Jeux.json', 'w') as outfile:
    json.dump(jeux, outfile)
print('données des jeux sauvegardé (Jeux.json)')
print('il y a ' + str(nErreur) + ' érreurs')
jeuDoublon
for jeu,paysJeu in paysManque.items():
  print('dans '+ jeu+ ' le pays '+ paysJeu+ ' est chelou')

1477 /1485
données des jeux sauvegardé (Jeux.json)
il y a 9 érreurs


In [ ]:
with open('lienErreur.json', 'w') as outfile:
    json.dump(lienErreur, outfile)
print('données des lien d\'Erreur sauvegardé (lienErreur.json)')

### Liste des Studios

In [28]:
linksStudio = []
for i in range(1,26):
  url = "https://lddm.psln.nl/studio?page=" + str(i)
  rep = requests.get(url)
  if rep.ok:
    soup = BeautifulSoup(rep.text, 'lxml')
    aList = soup.findAll('a', class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
    aList = aList[2:]
    for a in aList:
      link = a['href']
      linksStudio.append(link)
  else:
    print("problème à la page " + i)
    print(rep)
print("liste lien studios (linksStudio) : Ok")

liste lien studios (linksStudio) : Ok


In [29]:
studios = {}
lienErreur = []
nErreur = 0
boucle = 0
for link in linksStudio:
#for i in range(2):
  #link = linksStudio[i]
  rep = requests.get(link)
  if rep.ok:
    soup = BeautifulSoup(rep.text, 'lxml')
    nomStudio = re.sub(aRemplacer, caracSpe, soup.find('h1', class_="h5 card-header").text).strip()[:-4].strip()
    nat = pays[re.sub(aRemplacer, caracSpe, soup.find('h1', class_="h5 card-header").text).strip()[-4:].strip()]
    gameList = []
    games = soup.findAll("a", class_="list-group-item list-group-item-action d-flex justify-content-between align-items-center")
    for a in games:
      game = {'link' : a['href'],
              'name' : re.sub(aRemplacer, caracSpe, a.find('strong').text.strip())
              }
      gameList.append(game)
    studio = {
        'nomStudio' : nomStudio,
        'nat' : nat,
        'link' : link,
        'gameList' : gameList
        }
    clear_output(wait=True)
    boucle += 1
    print(boucle, '/742')
    studios[nomStudio] = studio
  else:
    nErreur += 1
    lienErreur.append(link)
    print("problème à la page " + link)
    print(rep)


with open('C:/Users/Jayma/Documents/Jupyter notebook/Json LDDM to Obsidian/Studios.json', 'w') as outfile:
    json.dump(studios, outfile)
print('données des studios sauvegardé (Studios.json)')
print('il y a ' + str(nErreur) + ' érreurs')

735 /742
données des studios sauvegardé (Studios.json)
il y a 0 érreurs


## Recomposition MarckDown

In [2]:
with open('C:/Users/Jayma/Documents/Jupyter notebook/Json LDDM to Obsidian/Episodes.json', 'r') as infile:
  episodes = json.load(infile)
with open('C:/Users/Jayma/Documents/Jupyter notebook/Json LDDM to Obsidian/Artistes.json', 'r') as infile:
  artistes = json.load(infile)
with open('C:/Users/Jayma/Documents/Jupyter notebook/Json LDDM to Obsidian/Jeux.json', 'r') as infile:
  jeux = json.load(infile)
with open('C:/Users/Jayma/Documents/Jupyter notebook/Json LDDM to Obsidian/Studios.json', 'r') as infile:
  studios = json.load(infile)

### Note des Episodes

In [28]:
if not os.path.exists('C:/Users/Jayma/Documents/Jupyter notebook/Episodes'):
 os.makedirs('C:/Users/Jayma/Documents/Jupyter notebook/Episodes')
for episode in episodes.values():
  textTrack = []
  for track in episode['trackListe']:
    artist = []
    for art in track['artist']:
      artist.append('[[Artistes/' + art+ '|'+ art+ ']]')
    textTrack.append(track['index'] + '. **' + track['name'] + '** - ' + ', '.join(artist) +('', ' *Reprise*')[track['reprise']]+ ('', '\n**[[Jeux/' + track['game'] + '|'+ track['game']+ ']] -** *[[Studios/' + track['studio'] + '|'+ track['studio']+ ']] - ' + track['dateJeu'] + '*')[bool(track['game'])])
  textRedige = (
      '# ' + episode['titre'] + '\n'
      '##### **saison n°' + episode['nSaison'] + ' - episode n°' + episode['nEpisode'] + '**\n'
      '📆*' + episode['datePubli'] + '* - ⏱' + episode['duree'] + '\n\n'
      + episode['text'] + '\n![[LDDM'+ episode['nEpisode']+ '.mp3]]\n'
      '### TrackList\n' +
      '\n\n'.join(textTrack)
  )
  with open('C:/Users/Jayma/Documents/Jupyter notebook/Episodes/' + episode['nEpisode'] + '- ' + episode['titre'] + '.md', 'w', encoding='utf8') as outfile:
    outfile.write(textRedige)
print('fini, tout sauvegardé dans le dossier Episodes')

fini, tout sauvegardé dans le dossier Episodes


### Note des Artistes

In [29]:
if not os.path.exists('C:/Users/Jayma/Documents/Jupyter notebook/Artistes'):
 os.makedirs('C:/Users/Jayma/Documents/Jupyter notebook/Artistes')
for artiste in artistes.values():
  textTrack = []
  for trackArt in artiste['trackList']:
    for trackEp in episodes[str(trackArt['indexEp'])]['trackListe']:
      if trackArt['name'] == trackEp['name'] and artiste['nomArt'] in trackEp['artist']:
        if trackEp['game']:
          textTrack.append('- **'+ trackArt['name']+ '** - [[Episodes/'+ trackArt['indexEp']+ '- '+ trackArt['nameEp']+ '|'+ trackArt['indexEp']+ '- '+ trackArt['nameEp']+ ']]'+ ('',' *Reprise*')[trackEp['reprise']]+ '\n**[[Jeux/'+ trackEp['game']+ '|'+ trackEp['game']+ ']] -** *[[Studios/'+ trackEp['studio']+ '|'+ trackEp['studio']+ ']] - '+ trackEp['dateJeu']+ '*')
        else:
          textTrack.append('- **'+ trackArt['name']+ '** - [[Episodes/'+ trackArt['indexEp']+ '- '+ trackArt['nameEp']+ '|'+ trackArt['indexEp']+ '- '+ trackArt['nameEp']+ ']]'+ ('',' *Reprise*')[trackEp['reprise']])
        break
  textSource = []
  for source in artiste['sourceListe']:
    textSource.append('- ['+ source['nom']+ ']('+ source['link']+ ')')
  if artiste['sexe']:
    if artiste['nomNom']:
      nomSexe = '##### **' + artiste['nomNom'] + '** - ' + artiste['sexe']
    else:
      nomSexe = '##### ' + artiste['sexe']
  else:
    if artiste['nomNom']:
      nomSexe = '##### **' + artiste['nomNom'] + '**'
    else:
      nomSexe = ''
  if artiste['dateNaisance']:
    naisance = '📆*' + artiste['dateNaisance'] + '*'
  else:
    naisance = ''
  if artiste['nationalite']:
    if artiste['dateNaisance']:
      nat = ' - #N/' + re.sub('\s', '_', artiste['nationalite'])
    else:
      nat = '#N/' + re.sub('\s', '_', artiste['nationalite'])
  else:
    nat = ''
  
  textRedige = (
      '# ' + artiste['nomArt'] + '\n'
      + nomSexe + '\n' +
      naisance + nat + '\n\n'+ ('', '\n### TrackList\n')[bool(textTrack)]+ 
      '\n\n'.join(textTrack)+ 
      ('', '\n### Sources\n')[bool(textSource)]+ 
      '\n'.join(textSource)
  )
  with open('C:/Users/Jayma/Documents/Jupyter notebook/Artistes/' + artiste['nomArt'] + '.md', 'w', encoding='utf8') as outfile:
    outfile.write(textRedige)
print('fini, tout sauvegardé dans le dossier Artistes')

fini, tout sauvegardé dans le dossier Artistes


In [30]:
artisteRelou = []
nomArtRelou = []
for artiste in artistes.keys():
  if re.search('\.\s', artiste):
    print('sa marche avec : '+ artiste)
    artisteRelou.append(artiste)
    nomArtRelou.append(artiste[re.search('\. ', artiste).end():])
print('fini')
print(nomArtRelou)

sa marche avec : B.B. Rights
sa marche avec : David S. Kates
sa marche avec : Donald S. Griffin
sa marche avec : G. Kill
sa marche avec : Isak J. Martinsson
sa marche avec : Kim M. Jensen
sa marche avec : T. Ashley Jenkins
sa marche avec : Tracy W. Bush
fini
['Rights', 'Kates', 'Griffin', 'Kill', 'Martinsson', 'Jensen', 'Ashley Jenkins', 'Bush']


### Notes des Jeux

In [21]:
if not os.path.exists('C:/Users/Jayma/Documents/Jupyter notebook/Jeux'):
 os.makedirs('C:/Users/Jayma/Documents/Jupyter notebook/Jeux')
for jeu in jeux.values():
  textTrack = []
  if bool(re.search('\s\(2\)$', jeu['nomJeu'])):
    jeutest = jeu['nomJeu'][:-4]
  else:
    jeutest = jeu['nomJeu']
  for trackJeu in jeu['trackList']:
    for trackEp in episodes[str(trackJeu['indexEp'])]['trackListe']:
      if trackJeu['name'] == trackEp['name'] and jeutest == trackEp['game']:
        artist = []
        for art in trackEp['artist']:
          artist.append('[[Artistes/'+ art+ '|'+ art+ ']]')
        textTrack.append('- **'+ trackJeu['name']+ '** - '+ ', '.join(artist)+ ' / *[[Episodes/'+ trackJeu['indexEp']+ '- '+ episodes[str(trackJeu['indexEp'])]['titre']+ '|'+ trackJeu['indexEp']+ '- '+ episodes[str(trackJeu['indexEp'])]['titre']+ ']]*'+ ('', ' *Reprise*')[trackEp['reprise']])
        break
  if jeu['studio']:
    if jeu['nat']:
      if jeu['datePubli']:
        SND = '##### **[[Studios/'+ jeu['studio']+ '|'+ jeu['studio']+ ']]** - #N/'+ re.sub('\s', '_', jeu['nat'])+ ' - *'+ jeu['datePubli']+ '*'
      else:
        SND = '##### **[[Studios/'+ jeu['studio']+ '|'+ jeu['studio']+ ']]** - #N/'+ re.sub('\s', '_', jeu['nat'])
    else:
      if jeu['datePubli']:
        SND = '##### **[[Studios/'+ jeu['studio']+ '|'+ jeu['studio']+ ']]** - *'+ jeu['datePubli']+ '*'
      else:
        SND = '##### **[[Studios/'+ jeu['studio']+ '|'+ jeu['studio']+ ']]**'
  else:
    if jeu['nat']:
      if jeu['datePubli']:
        SND = '##### #N/'+ re.sub('\s', '_', jeu['nat'])+ ' - *'+ jeu['datePubli']+ '*'
      else:
        SND = '##### #N/'+ re.sub('\s', '_', jeu['nat'])
    else:
      if jeu['datePubli']:
        SND = '##### *'+ jeu['datePubli']+ '*'
      else:
        SND = ''
  modes = []
  for mode in jeu['modeJeu']:
    modes.append('#M/'+ re.sub('\s', '_', mode))
  genres = ''
  for genre in jeu['genres']:
    genres = genres+ '- #G/'+ re.sub('\'', '', re.sub('\s', '_', genre))+ '\n'
  consoles = ''
  for console in jeu['consoles']:
    consoles = consoles+ '- #C/'+ re.sub('/', '-', re.sub('\s', '_', console))+ '\n'
  textRedige = (
      '# ' + jeu['nomJeu'] +  '\n'+
      SND + '\n'+ 
      ' - '.join(modes) + '\n'
      '###### Genres :\n'+ 
      genres+ 
      '\n###### Consoles :\n'+ 
      consoles+ 
      '\n### TrackList\n' +
      '\n\n'.join(textTrack)
  )
  with open('C:/Users/Jayma/Documents/Jupyter notebook/Jeux/' + jeu['nomJeu'] + '.md', 'w', encoding='utf8') as outfile:
    outfile.write(textRedige)
print('fini, tout sauvegardé dans le dossier Jeux')

fini, tout sauvegardé dans le dossier Jeux


In [17]:
textt = 'truccoqf (2)'
textt[:-4]

'truccoqf'

### Notes des Studios

In [36]:
if not os.path.exists('C:/Users/Jayma/Documents/Jupyter notebook/Studios'):
 os.makedirs('C:/Users/Jayma/Documents/Jupyter notebook/Studios')
for studio in studios.values():
  textJeux = []
  for jeu in studio['gameList']:
    artist = []
    morceauJeu = {}
    for morceau in jeux[jeu['name']]['trackList']:
      morceauJeu[morceau['name']] = morceau['indexEp']
    for morceauNom,ep in morceauJeu.items():
      for track in episodes[ep]['trackListe']:
        if track['name'] == morceauNom and studio['nomStudio'] == track['studio']:
          for art in track['artist']:
            if not ('[[Artistes/'+ art+ '|'+ art+ ']]' in artist or track['reprise']):
              artist.append('[[Artistes/'+ art+ '|'+ art+ ']]')
          break
    textJeux.append('- **[[Jeux/'+ jeu['name'] + '|'+ jeu['name']+ ']]** - *' + jeux[jeu['name']]['datePubli'][-4:]+ '*\n'+ ', '.join(artist))
  textRedige = (
      '# ' + studio['nomStudio'] + '\n'
      '##### #N/'+ re.sub('\s', '_', studio['nat']) + '\n\n'
      '### Liste de Jeux\n' +
      '\n\n'.join(textJeux)
  )
  with open('C:/Users/Jayma/Documents/Jupyter notebook/Studios/' + studio['nomStudio'] + '.md', 'w', encoding='utf8') as outfile:
    outfile.write(textRedige)
print('fini, tout sauvegardé dans le dossier Studios')

fini, tout sauvegardé dans le dossier Studios
